In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 确保openpyxl可用于Excel文件操作
try:
    import openpyxl
except ImportError:
    print("警告: openpyxl未安装，Excel功能可能不可用。请运行: pip install openpyxl")

# 设置中文字体和样式
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('default')

# 配置参数
CONFIG = {
    # 输入路径 - 模型训练结果的路径
    'input_path': r"D:\博一下\manuscript5-变Si组\模型训练_K45_物理引导特征工程_无贝叶斯优化-7.29.3最终_修复版",
    # 输出路径 - CDF分析结果保存路径
    'output_path': r"D:\博一下\manuscript5-变Si组\CDF分析8.1",
    # 文件后缀（适配修复版）
    'file_suffix': '_physics_guided_no_bayesian_fixed',
    # 分析参数
    'max_error': 50,  # 最大误差阈值（%）
    'top_models_count': 8,  # 详细对比图中显示的模型数量
    'error_thresholds': [5, 10, 15, 20, 25, 30],  # 误差阈值列表
    'figure_dpi': 300,  # 图片分辨率
    'enable_detailed_stats': True,  # 是否生成详细统计
}

def setup_output_directory():
    """创建输出目录"""
    output_dir = CONFIG['output_path']
    os.makedirs(output_dir, exist_ok=True)
    
    # 创建子目录
    subdirs = ['figures', 'data', 'statistics']
    for subdir in subdirs:
        os.makedirs(os.path.join(output_dir, subdir), exist_ok=True)
    
    print(f"✓ 输出目录已创建: {output_dir}")
    return output_dir

def load_model_data():
    """加载保存的模型和数据"""
    print("=== 正在加载模型数据 ===")
    input_path = CONFIG['input_path']
    suffix = CONFIG['file_suffix']
    
    try:
        # 加载模型列表
        models_file = os.path.join(input_path, f'best_models{suffix}.pkl')
        print(f"加载模型文件: {models_file}")
        with open(models_file, 'rb') as f:
            best_models = pickle.load(f)
        
        # 加载数据信息
        data_file = os.path.join(input_path, f'data_info{suffix}.pkl')
        print(f"加载数据文件: {data_file}")
        with open(data_file, 'rb') as f:
            data_info = pickle.load(f)
        
        # 加载特征名称
        features_file = os.path.join(input_path, f'features_name{suffix}.pkl')
        print(f"加载特征文件: {features_file}")
        with open(features_file, 'rb') as f:
            features_name = pickle.load(f)
        
        print(f"✓ 成功加载 {len(best_models)} 个模型")
        print(f"✓ 数据形状信息: X{data_info['X_all_scaled'].shape}, y{data_info['y_all_np'].shape}")
        print(f"✓ 特征数量: {len(features_name)}")
        
        return best_models, data_info, features_name
        
    except FileNotFoundError as e:
        print(f"❌ 文件未找到: {e}")
        print(f"请检查路径: {input_path}")
        raise
    except Exception as e:
        print(f"❌ 加载数据时发生错误: {e}")
        raise

def calculate_model_errors(best_models, data_info):
    """为每个模型计算预测误差和详细统计"""
    print("\n=== 正在计算模型预测误差 ===")
    
    model_errors = {}
    model_info = {}
    detailed_stats = {}
    
    for i, model_data in enumerate(best_models):
        model_key = model_data['model_key']
        method = model_data['method']
        ml_model = model_data['model']
        k = model_data['k']
        
        print(f"  [{i+1}/{len(best_models)}] 处理模型: {model_key}")
        
        try:
            # 获取模型实例和对应数据
            model_instance = model_data['model_instance']
            X_data = model_data['X_data']
            y_data = model_data['y_data']
            
            # 进行预测
            y_pred = model_instance.predict(X_data)
            
            # 计算各种误差指标
            absolute_errors = np.abs(y_data - y_pred)
            
            # 计算相对误差（百分比），避免除零
            y_true_safe = np.where(np.abs(y_data) < 1e-6, 1e-6, y_data)
            relative_errors = np.abs((y_data - y_pred) / y_true_safe) * 100
            
            # 计算标准化残差
            residuals = y_data - y_pred
            std_residuals = residuals / np.std(residuals) if np.std(residuals) > 0 else residuals
            
            # 存储误差
            model_errors[model_key] = relative_errors
            
            # 存储模型信息
            model_info[model_key] = {
                'method': method,
                'ml_model': ml_model,
                'k': k,
                'train_r2': model_data['train_r2'],
                'test_r2': model_data['test_r2'],
                'train_mae': model_data['train_mae'],
                'test_mae': model_data['test_mae'],
                'train_rmsle': model_data.get('train_rmsle', 0),
                'test_rmsle': model_data.get('test_rmsle', 0),
                'features': model_data['features'],
                'is_advanced': model_data['is_advanced'],
                'is_focus_combination': model_data['is_focus_combination'],
                'data_type': model_data.get('data_type', 'Unknown')
            }
            
            # 计算详细统计信息
            if CONFIG['enable_detailed_stats']:
                detailed_stats[model_key] = {
                    'y_true': y_data,
                    'y_pred': y_pred,
                    'absolute_errors': absolute_errors,
                    'relative_errors': relative_errors,
                    'residuals': residuals,
                    'std_residuals': std_residuals,
                    'rmse': np.sqrt(mean_squared_error(y_data, y_pred)),
                    'mae': mean_absolute_error(y_data, y_pred),
                    'r2': r2_score(y_data, y_pred),
                    'mean_relative_error': np.mean(relative_errors),
                    'median_relative_error': np.median(relative_errors),
                    'std_relative_error': np.std(relative_errors),
                    'max_relative_error': np.max(relative_errors),
                    'min_relative_error': np.min(relative_errors)
                }
            
            print(f"    ✓ R²={model_data['test_r2']:.4f}, MAE={model_data['test_mae']:.4f}, "
                  f"平均相对误差={np.mean(relative_errors):.2f}%")
            
        except Exception as e:
            print(f"    ❌ 处理模型 {model_key} 时出错: {e}")
            continue
    
    print(f"✓ 成功处理 {len(model_errors)} 个模型")
    return model_errors, model_info, detailed_stats

def save_data_files(model_errors, model_info, detailed_stats, output_dir):
    """保存所有数据文件"""
    print("\n=== 保存数据文件 ===")
    data_dir = os.path.join(output_dir, 'data')
    excel_dir = os.path.join(output_dir, 'excel_for_origin')
    os.makedirs(excel_dir, exist_ok=True)
    
    # 1. 保存模型误差数据
    errors_file = os.path.join(data_dir, 'model_relative_errors.pkl')
    with open(errors_file, 'wb') as f:
        pickle.dump(model_errors, f)
    print(f"✓ 保存相对误差数据: {os.path.basename(errors_file)}")
    
    # 2. 保存模型信息
    info_file = os.path.join(data_dir, 'model_info.pkl')
    with open(info_file, 'wb') as f:
        pickle.dump(model_info, f)
    print(f"✓ 保存模型信息: {os.path.basename(info_file)}")
    
    # 3. 保存详细统计数据
    if detailed_stats:
        stats_file = os.path.join(data_dir, 'detailed_statistics.pkl')
        with open(stats_file, 'wb') as f:
            pickle.dump(detailed_stats, f)
        print(f"✓ 保存详细统计: {os.path.basename(stats_file)}")
    
    # 4. 保存为CSV格式（方便查看）
    # 创建汇总表
    summary_data = []
    for model_key, info in model_info.items():
        features_str = ', '.join(info['features']) if isinstance(info['features'], list) else str(info['features'])
        
        row = {
            'Model_Key': model_key,
            'Method': info['method'],
            'ML_Model': info['ml_model'],
            'K': info['k'],
            'Train_R2': info['train_r2'],
            'Test_R2': info['test_r2'],
            'Train_MAE': info['train_mae'],
            'Test_MAE': info['test_mae'],
            'Train_RMSLE': info['train_rmsle'],
            'Test_RMSLE': info['test_rmsle'],
            'Features': features_str,
            'Is_Advanced': info['is_advanced'],
            'Is_Focus_Combination': info['is_focus_combination'],
            'Data_Type': info['data_type']
        }
        
        # 添加误差统计
        if model_key in model_errors:
            errors = model_errors[model_key]
            row.update({
                'Mean_Relative_Error': np.mean(errors),
                'Median_Relative_Error': np.median(errors),
                'Std_Relative_Error': np.std(errors),
                'Max_Relative_Error': np.max(errors),
                'Min_Relative_Error': np.min(errors)
            })
            
            # 添加不同阈值下的累积频率
            for threshold in CONFIG['error_thresholds']:
                row[f'Freq_Below_{threshold}pct'] = np.mean(errors <= threshold)
        
        summary_data.append(row)
    
    # 保存汇总CSV
    summary_df = pd.DataFrame(summary_data)
    summary_df = summary_df.sort_values('Test_R2', ascending=False)
    summary_csv = os.path.join(data_dir, 'model_summary_with_errors.csv')
    summary_df.to_csv(summary_csv, index=False, encoding='utf-8')
    print(f"✓ 保存汇总CSV: {os.path.basename(summary_csv)}")
    
    # 5. 保存每个模型的详细误差数据
    errors_detail_dir = os.path.join(data_dir, 'individual_errors')
    os.makedirs(errors_detail_dir, exist_ok=True)
    
    for model_key, errors in model_errors.items():
        error_df = pd.DataFrame({
            'Sample_Index': range(len(errors)),
            'Relative_Error_Percent': errors
        })
        error_file = os.path.join(errors_detail_dir, f'{model_key}_errors.csv')
        error_df.to_csv(error_file, index=False)
    
    print(f"✓ 保存个体误差文件到: individual_errors/")
    
    # 6. 为Origin软件准备Excel数据
    print("  为Origin软件准备Excel数据...")
    save_excel_for_origin(model_errors, model_info, detailed_stats, excel_dir)
    
    return summary_df

def save_excel_for_origin(model_errors, model_info, detailed_stats, excel_dir):
    """为Origin软件保存所有可视化数据的Excel文件"""
    print("    正在生成Origin可用的Excel文件...")
    
    # 1. 保存所有模型的CDF数据 - 主要数据表
    print("    生成CDF数据表...")
    cdf_excel_file = os.path.join(excel_dir, '01_All_Models_CDF_Data.xlsx')
    
    with pd.ExcelWriter(cdf_excel_file, engine='openpyxl') as writer:
        # 工作表1: 所有模型的CDF原始数据
        all_cdf_data = {}
        model_labels = []
        
        for model_key, errors in model_errors.items():
            info = model_info[model_key]
            errors_filtered = errors[errors <= CONFIG['max_error']]
            errors_sorted = np.sort(errors_filtered)
            
            if len(errors_sorted) > 0:
                cumulative_prob = np.arange(1, len(errors_sorted) + 1) / len(errors_sorted)
                
                # 创建标签
                advanced_mark = "★" if info['is_advanced'] else ""
                focus_mark = "🎯" if info['is_focus_combination'] else ""
                label = f"{advanced_mark}{focus_mark}{info['method']}+{info['ml_model']}_K{info['k']}"
                model_labels.append(label)
                
                # 统一误差范围到0-50，插值到相同的x轴
                x_common = np.linspace(0, CONFIG['max_error'], 1000)
                y_interpolated = np.interp(x_common, errors_sorted, cumulative_prob)
                
                all_cdf_data[f'{label}_Error'] = x_common
                all_cdf_data[f'{label}_CumProb'] = y_interpolated
        
        # 保存CDF插值数据
        cdf_df = pd.DataFrame(all_cdf_data)
        cdf_df.to_excel(writer, sheet_name='CDF_Interpolated_Data', index=False)
        
        # 工作表2: 原始CDF数据点（每个模型一列）
        max_length = max(len(errors[errors <= CONFIG['max_error']]) for errors in model_errors.values())
        raw_cdf_data = {'Common_Index': range(max_length)}
        
        for model_key, errors in model_errors.items():
            info = model_info[model_key]
            errors_filtered = errors[errors <= CONFIG['max_error']]
            errors_sorted = np.sort(errors_filtered)
            
            if len(errors_sorted) > 0:
                cumulative_prob = np.arange(1, len(errors_sorted) + 1) / len(errors_sorted)
                label = f"{info['method']}+{info['ml_model']}_K{info['k']}"
                
                # 填充到最大长度
                padded_errors = np.full(max_length, np.nan)
                padded_probs = np.full(max_length, np.nan)
                
                padded_errors[:len(errors_sorted)] = errors_sorted
                padded_probs[:len(cumulative_prob)] = cumulative_prob
                
                raw_cdf_data[f'{label}_Error'] = padded_errors
                raw_cdf_data[f'{label}_CumProb'] = padded_probs
        
        raw_cdf_df = pd.DataFrame(raw_cdf_data)
        raw_cdf_df.to_excel(writer, sheet_name='CDF_Raw_Data_Points', index=False)
        
        # 工作表3: 模型信息和统计
        model_stats = []
        for model_key, info in model_info.items():
            if model_key in model_errors:
                errors = model_errors[model_key]
                model_stats.append({
                    'Model_Key': model_key,
                    'Display_Name': f"{info['method']}+{info['ml_model']}_K{info['k']}",
                    'Method': info['method'],
                    'ML_Model': info['ml_model'],
                    'K': info['k'],
                    'Test_R2': info['test_r2'],
                    'Test_MAE': info['test_mae'],
                    'Is_Advanced': info['is_advanced'],
                    'Is_Focus_Combination': info['is_focus_combination'],
                    'Mean_Error': np.mean(errors),
                    'Median_Error': np.median(errors),
                    'Std_Error': np.std(errors),
                    'Sample_Count': len(errors),
                    'Freq_5pct': np.mean(errors <= 5),
                    'Freq_10pct': np.mean(errors <= 10),
                    'Freq_20pct': np.mean(errors <= 20),
                    'Color_Code': f'Color_{list(model_errors.keys()).index(model_key) + 1}'
                })
        
        stats_df = pd.DataFrame(model_stats)
        stats_df = stats_df.sort_values('Test_R2', ascending=False)
        stats_df.to_excel(writer, sheet_name='Model_Info_and_Stats', index=False)
    
    print(f"    ✓ 主CDF数据表: {os.path.basename(cdf_excel_file)}")
    
    # 2. 保存顶级模型对比数据
    print("    生成顶级模型对比数据...")
    top_models_file = os.path.join(excel_dir, '02_Top_Models_Comparison.xlsx')
    
    sorted_models = sorted(model_errors.items(), 
                          key=lambda x: model_info[x[0]]['test_r2'], 
                          reverse=True)
    top_models = sorted_models[:CONFIG['top_models_count']]
    
    with pd.ExcelWriter(top_models_file, engine='openpyxl') as writer:
        # 顶级模型CDF数据
        x_common = np.linspace(0, 35, 500)
        top_cdf_data = {'Error_Percent': x_common}
        
        for i, (model_key, errors) in enumerate(top_models):
            info = model_info[model_key]
            errors_filtered = errors[errors <= 35]
            errors_sorted = np.sort(errors_filtered)
            
            if len(errors_sorted) > 0:
                cumulative_prob = np.arange(1, len(errors_sorted) + 1) / len(errors_sorted)
                y_interpolated = np.interp(x_common, errors_sorted, cumulative_prob)
                
                method_short = info['method'].replace('F-Regression', 'F-Reg').replace('Mutual Information', 'MutInfo')
                model_short = info['ml_model'].replace('GradientBoosting', 'GB').replace('RandomForest', 'RF')
                advanced_mark = "★" if info['is_advanced'] else ""
                
                label = f"{advanced_mark}{method_short}+{model_short}"
                top_cdf_data[f'{label}_CumProb'] = y_interpolated
                top_cdf_data[f'{label}_R2'] = [info['test_r2']] * len(x_common)  # 用于标注
        
        top_cdf_df = pd.DataFrame(top_cdf_data)
        top_cdf_df.to_excel(writer, sheet_name='Top_Models_CDF', index=False)
        
        # 顶级模型统计对比
        top_stats = []
        for model_key, errors in top_models:
            info = model_info[model_key]
            top_stats.append({
                'Rank': len(top_stats) + 1,
                'Model': f"{info['method']}+{info['ml_model']}",
                'K': info['k'],
                'Test_R2': info['test_r2'],
                'Test_MAE': info['test_mae'],
                'Mean_Error': np.mean(errors),
                'Freq_5pct': np.mean(errors <= 5),
                'Freq_10pct': np.mean(errors <= 10),
                'Freq_20pct': np.mean(errors <= 20),
                'Is_Advanced': info['is_advanced'],
                'LineStyle': ['solid', 'dashed', 'dashdot', 'dotted'][len(top_stats) % 4]
            })
        
        top_stats_df = pd.DataFrame(top_stats)
        top_stats_df.to_excel(writer, sheet_name='Top_Models_Stats', index=False)
    
    print(f"    ✓ 顶级模型对比: {os.path.basename(top_models_file)}")
    
    # 3. 保存误差分布直方图数据
    print("    生成误差分布直方图数据...")
    histogram_file = os.path.join(excel_dir, '03_Error_Distribution_Histograms.xlsx')
    
    with pd.ExcelWriter(histogram_file, engine='openpyxl') as writer:
        # 最优模型的误差分布
        best_model_key = max(model_errors.keys(), key=lambda x: model_info[x]['test_r2'])
        best_errors = model_errors[best_model_key]
        best_errors_filtered = best_errors[best_errors <= 30]
        
        # 计算直方图
        counts, bin_edges = np.histogram(best_errors_filtered, bins=30)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        
        hist_data = {
            'Bin_Centers': bin_centers,
            'Counts': counts,
            'Bin_Edges_Left': bin_edges[:-1],
            'Bin_Edges_Right': bin_edges[1:],
            'Bin_Width': bin_edges[1] - bin_edges[0]
        }
        
        # 添加统计线数据
        hist_data['Mean_Error'] = [np.mean(best_errors_filtered)] * len(bin_centers)
        hist_data['Median_Error'] = [np.median(best_errors_filtered)] * len(bin_centers)
        hist_data['Max_Count_for_Lines'] = [max(counts)] * len(bin_centers)
        
        hist_df = pd.DataFrame(hist_data)
        hist_df.to_excel(writer, sheet_name='Best_Model_Histogram', index=False)
        
        # 所有模型的直方图数据（简化版）
        all_hist_data = {'Error_Range': bin_centers}
        for model_key, errors in list(model_errors.items())[:8]:  # 只保存前8个模型
            info = model_info[model_key]
            errors_filtered = errors[errors <= 30]
            
            if len(errors_filtered) > 0:
                counts, _ = np.histogram(errors_filtered, bins=bin_edges)
                model_name = f"{info['method']}+{info['ml_model']}_K{info['k']}"
                all_hist_data[f'{model_name}_Counts'] = counts
        
        all_hist_df = pd.DataFrame(all_hist_data)
        all_hist_df.to_excel(writer, sheet_name='All_Models_Histograms', index=False)
    
    print(f"    ✓ 误差分布直方图: {os.path.basename(histogram_file)}")
    
    # 4. 保存雷达图数据
    print("    生成雷达图数据...")
    radar_file = os.path.join(excel_dir, '04_Radar_Chart_Data.xlsx')
    
    with pd.ExcelWriter(radar_file, engine='openpyxl') as writer:
        # 选择前6个模型
        radar_models = sorted_models[:6]
        metrics = ['Test_R2', 'Freq_5pct', 'Freq_10pct', 'Freq_20pct', 'Low_Error_Rate']
        metric_labels = ['R²', '5%误差率', '10%误差率', '20%误差率', '低误差率']
        
        # 雷达图数据
        radar_data = []
        for model_key, _ in radar_models:
            info = model_info[model_key]
            errors = model_errors[model_key]
            
            row = {
                'Model': f"{info['method']}+{info['ml_model']}_K{info['k']}",
                'Test_R2': info['test_r2'],
                'Freq_5pct': np.mean(errors <= 5),
                'Freq_10pct': np.mean(errors <= 10),
                'Freq_20pct': np.mean(errors <= 20),
                'Low_Error_Rate': np.mean(errors <= 15),
                'Is_Advanced': info['is_advanced'],
                'Is_Focus': info['is_focus_combination']
            }
            radar_data.append(row)
        
        radar_df = pd.DataFrame(radar_data)
        radar_df.to_excel(writer, sheet_name='Radar_Data', index=False)
        
        # 雷达图配置（角度等）
        angles_deg = np.linspace(0, 360, len(metrics) + 1)[:-1]  # 不包含360度点
        angles_rad = np.deg2rad(angles_deg)
        
        config_data = {
            'Metric': metric_labels,
            'Angle_Degrees': angles_deg,
            'Angle_Radians': angles_rad,
            'Max_Value': [1.0] * len(metrics),  # 所有指标最大值为1
            'Min_Value': [0.0] * len(metrics),  # 所有指标最小值为0
        }
        
        config_df = pd.DataFrame(config_data)
        config_df.to_excel(writer, sheet_name='Radar_Config', index=False)
    
    print(f"    ✓ 雷达图数据: {os.path.basename(radar_file)}")
    
    # 5. 保存每个模型的详细分析数据
    print("    生成单独模型分析数据...")
    individual_dir = os.path.join(excel_dir, 'individual_models')
    os.makedirs(individual_dir, exist_ok=True)
    
    for i, (model_key, errors) in enumerate(list(model_errors.items())[:5]):  # 只为前5个模型生成详细数据
        info = model_info[model_key]
        stats = detailed_stats.get(model_key, {})
        
        print(f"      处理模型 {i+1}/5: {model_key}")
        
        try:
            safe_filename = model_key.replace('/', '_').replace('\\', '_').replace(':', '_')
            individual_file = os.path.join(individual_dir, f'{safe_filename}_analysis.xlsx')
            
            with pd.ExcelWriter(individual_file, engine='openpyxl') as writer:
                # CDF数据
                errors_filtered = errors[errors <= 40]
                errors_sorted = np.sort(errors_filtered)
                if len(errors_sorted) > 0:
                    cumulative_prob = np.arange(1, len(errors_sorted) + 1) / len(errors_sorted)
                    
                    cdf_data = {
                        'Error_Percent': errors_sorted,
                        'Cumulative_Probability': cumulative_prob,
                        'Mean_Line_X': [np.mean(errors_filtered)] * len(errors_sorted),
                        'Mean_Line_Y': np.linspace(0, 1, len(errors_sorted)),
                        'Median_Line_X': [np.median(errors_filtered)] * len(errors_sorted),
                        'Median_Line_Y': np.linspace(0, 1, len(errors_sorted))
                    }
                    
                    cdf_df = pd.DataFrame(cdf_data)
                    cdf_df.to_excel(writer, sheet_name='CDF_Data', index=False)
                
                # 直方图数据
                counts, bin_edges = np.histogram(errors_filtered, bins=25)
                bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
                
                hist_data = {
                    'Bin_Centers': bin_centers,
                    'Counts': counts,
                    'Bin_Left': bin_edges[:-1],
                    'Bin_Right': bin_edges[1:],
                    'Mean_Value': [np.mean(errors_filtered)] * len(bin_centers),
                    'Median_Value': [np.median(errors_filtered)] * len(bin_centers)
                }
                
                hist_df = pd.DataFrame(hist_data)
                hist_df.to_excel(writer, sheet_name='Histogram_Data', index=False)
                
                # 预测vs实际散点图数据
                if 'y_true' in stats and 'y_pred' in stats:
                    y_true = stats['y_true']
                    y_pred = stats['y_pred']
                    
                    # 散点数据
                    scatter_data = {
                        'Actual_Values': y_true,
                        'Predicted_Values': y_pred,
                        'R2_Value': [stats.get('r2', 0)] * len(y_true)
                    }
                    
                    scatter_df = pd.DataFrame(scatter_data)
                    scatter_df.to_excel(writer, sheet_name='Prediction_Scatter', index=False)
                    
                    # 完美预测线数据（单独工作表）
                    min_val = min(np.min(y_true), np.min(y_pred))
                    max_val = max(np.max(y_true), np.max(y_pred))
                    perfect_line_data = {
                        'Perfect_Line_X': [min_val, max_val],
                        'Perfect_Line_Y': [min_val, max_val]
                    }
                    
                    perfect_line_df = pd.DataFrame(perfect_line_data)
                    perfect_line_df.to_excel(writer, sheet_name='Perfect_Prediction_Line', index=False)
                
                # 残差图数据
                if 'residuals' in stats:
                    residuals = stats['residuals']
                    y_pred = stats['y_pred']
                    
                    # 残差散点数据
                    residual_data = {
                        'Predicted_Values': y_pred,
                        'Residuals': residuals
                    }
                    
                    residual_df = pd.DataFrame(residual_data)
                    residual_df.to_excel(writer, sheet_name='Residual_Data', index=False)
                    
                    # 零线数据（单独工作表）
                    zero_line_data = {
                        'Zero_Line_X': [np.min(y_pred), np.max(y_pred)],
                        'Zero_Line_Y': [0, 0]
                    }
                    
                    zero_line_df = pd.DataFrame(zero_line_data)
                    zero_line_df.to_excel(writer, sheet_name='Zero_Reference_Line', index=False)
                
                # 模型信息
                model_info_data = {
                    'Property': ['Model_Key', 'Method', 'ML_Model', 'K', 'Test_R2', 'Test_MAE', 
                               'Mean_Error', 'Median_Error', 'Std_Error', 'Sample_Count'],
                    'Value': [model_key, info['method'], info['ml_model'], info['k'], 
                             info['test_r2'], info['test_mae'], np.mean(errors), 
                             np.median(errors), np.std(errors), len(errors)]
                }
                
                info_df = pd.DataFrame(model_info_data)
                info_df.to_excel(writer, sheet_name='Model_Info', index=False)
                
        except Exception as e:
            print(f"        ❌ 处理模型 {model_key} 时出错: {e}")
            continue
    
    print(f"    ✓ 单独模型分析数据: individual_models/ (前5个模型)")
    
    # 6. 创建Origin绘图指南
    print("    生成Origin绘图指南...")
    guide_file = os.path.join(excel_dir, '00_Origin_Plotting_Guide.xlsx')
    
    with pd.ExcelWriter(guide_file, engine='openpyxl') as writer:
        # 文件说明
        guide_data = {
            'File_Name': [
                '01_All_Models_CDF_Data.xlsx',
                '02_Top_Models_Comparison.xlsx', 
                '03_Error_Distribution_Histograms.xlsx',
                '04_Radar_Chart_Data.xlsx',
                'individual_models/[model]_analysis.xlsx'
            ],
            'Chart_Type': [
                'Line Chart (CDF)',
                'Line Chart (Top Models)',
                'Histogram',
                'Radar Chart',
                'Multiple Charts'
            ],
            'Key_Worksheet': [
                'CDF_Interpolated_Data',
                'Top_Models_CDF',
                'Best_Model_Histogram',
                'Radar_Data',
                'Multiple sheets'
            ],
            'X_Column': [
                '[Model]_Error columns',
                'Error_Percent',
                'Bin_Centers',
                'Radar chart (polar)',
                'Varies by chart'
            ],
            'Y_Column': [
                '[Model]_CumProb columns',
                '[Model]_CumProb columns',
                'Counts',
                'Metric values',
                'Varies by chart'
            ],
            'Notes': [
                'Multiple lines, one per model',
                'Top performing models only',
                'Add mean/median lines',
                'Use polar coordinates',
                'Individual model analysis'
            ]
        }
        
        guide_df = pd.DataFrame(guide_data)
        guide_df.to_excel(writer, sheet_name='Plotting_Guide', index=False)
        
        # 颜色和样式建议
        color_data = []
        for i, (model_key, info) in enumerate(list(model_info.items())[:15]):
            color_data.append({
                'Model': f"{info['method']}+{info['ml_model']}_K{info['k']}",
                'Suggested_Color': f'RGB({50 + i*15}, {100 + i*10}, {200 - i*8})',
                'Line_Style': ['Solid', 'Dash', 'Dot', 'DashDot'][i % 4],
                'Line_Width': 2 if info['is_focus_combination'] else 1.5,
                'Priority': 'High' if info['is_focus_combination'] else 'Normal'
            })
        
        color_df = pd.DataFrame(color_data)
        color_df.to_excel(writer, sheet_name='Style_Suggestions', index=False)
    
    print(f"    ✓ Origin绘图指南: {os.path.basename(guide_file)}")
    print(f"    ✅ 所有Excel数据文件已生成完成!")

def create_comprehensive_cdf_plot(model_errors, model_info, output_dir):
    """创建综合CDF分析图"""
    print("\n=== 创建综合CDF分析图 ===")
    
    # 设置图形
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))
    
    # 定义颜色映射
    colors = plt.cm.Set3(np.linspace(0, 1, len(model_errors)))
    
    # 统计数据存储
    stats_data = []
    
    # 子图1: 完整CDF曲线
    print("  绘制完整CDF曲线...")
    for i, (model_key, errors) in enumerate(model_errors.items()):
        info = model_info[model_key]
        
        # 过滤异常值
        errors_filtered = errors[errors <= CONFIG['max_error']]
        errors_sorted = np.sort(errors_filtered)
        
        # 计算累积概率
        n = len(errors_sorted)
        if n > 0:
            cumulative_prob = np.arange(1, n + 1) / n
            
            # 创建标签
            advanced_mark = "★" if info['is_advanced'] else ""
            focus_mark = "🎯" if info['is_focus_combination'] else ""
            label = f"{advanced_mark}{focus_mark}{info['method']}+{info['ml_model']} (K={info['k']})"
            
            # 绘制CDF曲线
            ax1.plot(errors_sorted, cumulative_prob, 
                    label=label, linewidth=2, color=colors[i], alpha=0.8)
            
            # 计算统计数据
            for threshold in CONFIG['error_thresholds']:
                freq = np.mean(errors_filtered <= threshold) if len(errors_filtered) > 0 else 0
                stats_data.append({
                    'Model': model_key,
                    'Method': info['method'],
                    'ML_Model': info['ml_model'],
                    'K': info['k'],
                    'Test_R2': info['test_r2'],
                    'Threshold': threshold,
                    'Frequency': freq,
                    'Advanced': info['is_advanced'],
                    'Focus': info['is_focus_combination']
                })
    
    # 设置子图1
    ax1.set_xlabel('相对误差 (%)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('累积频率', fontsize=12, fontweight='bold')
    ax1.set_title('所有模型预测误差累积分布 (CDF)', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    ax1.set_xlim(0, CONFIG['max_error'])
    ax1.set_ylim(0, 1)
    
    # 添加参考线
    for threshold in [5, 10, 20]:
        ax1.axvline(x=threshold, color='red', linestyle='--', alpha=0.5)
        ax1.text(threshold, 0.95, f'{threshold}%', rotation=90, 
                verticalalignment='top', color='red', fontsize=9)
    
    # 子图2: 顶级模型详细对比
    print("  绘制顶级模型详细对比...")
    sorted_models = sorted(model_errors.items(), 
                          key=lambda x: model_info[x[0]]['test_r2'], 
                          reverse=True)
    top_models = sorted_models[:CONFIG['top_models_count']]
    
    line_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']
    linestyles = ['-', '--', '-.', ':', '-', '--', '-.', ':']
    
    for i, (model_key, errors) in enumerate(top_models):
        info = model_info[model_key]
        errors_filtered = errors[errors <= 35]
        errors_sorted = np.sort(errors_filtered)
        
        n = len(errors_sorted)
        if n > 0:
            cumulative_prob = np.arange(1, n + 1) / n
            
            method_short = info['method'].replace('F-Regression', 'F-Reg').replace('Mutual Information', 'MutInfo')
            model_short = info['ml_model'].replace('GradientBoosting', 'GB').replace('RandomForest', 'RF')
            advanced_mark = "★" if info['is_advanced'] else ""
            
            label = f"{advanced_mark}{method_short}+{model_short} (R²={info['test_r2']:.3f})"
            
            ax2.plot(errors_sorted, cumulative_prob, 
                    label=label, 
                    color=line_colors[i % len(line_colors)], 
                    linestyle=linestyles[i % len(linestyles)],
                    linewidth=2.5, alpha=0.8)
    
    ax2.set_xlabel('相对误差 (%)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('累积频率', fontsize=12, fontweight='bold')
    ax2.set_title(f'顶级{CONFIG["top_models_count"]}个模型误差对比', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.legend(loc='lower right', fontsize=9)
    ax2.set_xlim(0, 35)
    ax2.set_ylim(0, 1)
    
    # 添加参考线
    for threshold in [5, 10, 20]:
        ax2.axvline(x=threshold, color='gray', linestyle=':', alpha=0.7)
        ax2.text(threshold, 0.02, f'{threshold}%', horizontalalignment='center', 
                verticalalignment='bottom', color='gray', fontsize=9)
    
    # 子图3: 误差分布直方图（以最优模型为例）
    print("  绘制误差分布直方图...")
    best_model_key = max(model_errors.keys(), key=lambda x: model_info[x]['test_r2'])
    best_errors = model_errors[best_model_key]
    
    ax3.hist(best_errors[best_errors <= 30], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    ax3.axvline(np.mean(best_errors), color='red', linestyle='--', linewidth=2, label=f'平均值: {np.mean(best_errors):.2f}%')
    ax3.axvline(np.median(best_errors), color='orange', linestyle='--', linewidth=2, label=f'中位数: {np.median(best_errors):.2f}%')
    ax3.set_xlabel('相对误差 (%)', fontsize=12, fontweight='bold')
    ax3.set_ylabel('频次', fontsize=12, fontweight='bold')
    ax3.set_title(f'最优模型误差分布\n({model_info[best_model_key]["method"]}+{model_info[best_model_key]["ml_model"]})', 
                 fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.legend()
    
    # 子图4: 模型性能对比雷达图
    print("  绘制模型性能雷达图...")
    
    # 选择前6个模型进行雷达图对比
    radar_models = sorted_models[:6]
    metrics = ['Test_R2', 'Freq_5pct', 'Freq_10pct', 'Freq_20pct', 'Low_Error_Rate']
    
    # 计算指标
    radar_data = []
    model_names = []
    
    for model_key, _ in radar_models:
        info = model_info[model_key]
        errors = model_errors[model_key]
        
        # 计算各项指标（归一化到0-1）
        test_r2 = info['test_r2']
        freq_5 = np.mean(errors <= 5)
        freq_10 = np.mean(errors <= 10) 
        freq_20 = np.mean(errors <= 20)
        low_error_rate = np.mean(errors <= 15)  # 低误差率
        
        radar_data.append([test_r2, freq_5, freq_10, freq_20, low_error_rate])
        
        # 简化模型名称
        method_short = info['method'][:5]
        model_short = info['ml_model'][:5]
        model_names.append(f"{method_short}+{model_short}")
    
    # 创建雷达图
    angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
    angles += angles[:1]  # 闭合
    
    ax4 = plt.subplot(2, 2, 4, projection='polar')
    
    for i, (data, name) in enumerate(zip(radar_data, model_names)):
        values = data + data[:1]  # 闭合数据
        ax4.plot(angles, values, 'o-', linewidth=2, label=name, color=line_colors[i % len(line_colors)])
        ax4.fill(angles, values, alpha=0.25, color=line_colors[i % len(line_colors)])
    
    ax4.set_xticks(angles[:-1])
    ax4.set_xticklabels(['R²', '5%误差', '10%误差', '20%误差', '低误差率'])
    ax4.set_ylim(0, 1)
    ax4.set_title('模型综合性能对比', fontsize=14, fontweight='bold', pad=20)
    ax4.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    ax4.grid(True)
    
    plt.tight_layout()
    
    # 保存图形
    output_path = os.path.join(output_dir, 'figures', 'comprehensive_cdf_analysis.png')
    plt.savefig(output_path, dpi=CONFIG['figure_dpi'], bbox_inches='tight', 
                facecolor='white', edgecolor='none')
    print(f"✓ 综合CDF图已保存: {os.path.basename(output_path)}")
    
    plt.close()
    return stats_data

def create_individual_model_plots(model_errors, model_info, detailed_stats, output_dir):
    """为每个模型创建单独的分析图"""
    print("\n=== 创建单独模型分析图 ===")
    
    individual_dir = os.path.join(output_dir, 'figures', 'individual_models')
    os.makedirs(individual_dir, exist_ok=True)
    
    for i, (model_key, errors) in enumerate(model_errors.items()):
        info = model_info[model_key]
        stats = detailed_stats.get(model_key, {})
        
        print(f"  [{i+1}/{len(model_errors)}] 创建 {model_key} 的分析图...")
        
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
        
        # 子图1: CDF曲线
        errors_filtered = errors[errors <= 40]
        errors_sorted = np.sort(errors_filtered)
        n = len(errors_sorted)
        if n > 0:
            cumulative_prob = np.arange(1, n + 1) / n
            ax1.plot(errors_sorted, cumulative_prob, linewidth=3, color='blue')
            ax1.fill_between(errors_sorted, cumulative_prob, alpha=0.3, color='lightblue')
        
        ax1.set_xlabel('相对误差 (%)', fontweight='bold')
        ax1.set_ylabel('累积频率', fontweight='bold')
        ax1.set_title('误差累积分布函数 (CDF)', fontweight='bold')
        ax1.grid(True, alpha=0.3)
        ax1.set_xlim(0, 40)
        
        # 添加统计线
        mean_error = np.mean(errors_filtered)
        median_error = np.median(errors_filtered)
        ax1.axvline(mean_error, color='red', linestyle='--', label=f'平均值: {mean_error:.2f}%')
        ax1.axvline(median_error, color='orange', linestyle='--', label=f'中位数: {median_error:.2f}%')
        ax1.legend()
        
        # 子图2: 误差直方图
        ax2.hist(errors_filtered, bins=25, alpha=0.7, color='lightgreen', edgecolor='black')
        ax2.axvline(mean_error, color='red', linestyle='--', linewidth=2)
        ax2.axvline(median_error, color='orange', linestyle='--', linewidth=2)
        ax2.set_xlabel('相对误差 (%)', fontweight='bold')
        ax2.set_ylabel('频次', fontweight='bold')
        ax2.set_title('误差分布直方图', fontweight='bold')
        ax2.grid(True, alpha=0.3)
        
        # 子图3: 预测vs实际值散点图
        if 'y_true' in stats and 'y_pred' in stats:
            y_true = stats['y_true']
            y_pred = stats['y_pred']
            
            ax3.scatter(y_true, y_pred, alpha=0.6, color='purple')
            
            # 添加完美预测线
            min_val = min(np.min(y_true), np.min(y_pred))
            max_val = max(np.max(y_true), np.max(y_pred))
            ax3.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='完美预测')
            
            ax3.set_xlabel('实际值', fontweight='bold')
            ax3.set_ylabel('预测值', fontweight='bold')
            ax3.set_title(f'预测vs实际值 (R²={stats.get("r2", 0):.4f})', fontweight='bold')
            ax3.grid(True, alpha=0.3)
            ax3.legend()
        
        # 子图4: 残差图
        if 'residuals' in stats:
            residuals = stats['residuals']
            y_pred = stats['y_pred']
            
            ax4.scatter(y_pred, residuals, alpha=0.6, color='brown')
            ax4.axhline(y=0, color='red', linestyle='--', linewidth=2)
            ax4.set_xlabel('预测值', fontweight='bold')
            ax4.set_ylabel('残差', fontweight='bold')
            ax4.set_title('残差分布图', fontweight='bold')
            ax4.grid(True, alpha=0.3)
        
        # 添加总标题和信息
        advanced_mark = "★高级方法 " if info['is_advanced'] else ""
        focus_mark = "🎯重点组合 " if info['is_focus_combination'] else ""
        suptitle = f"{advanced_mark}{focus_mark}{info['method']} + {info['ml_model']} (K={info['k']})\n"
        suptitle += f"Test R²={info['test_r2']:.4f}, MAE={info['test_mae']:.4f}, 平均相对误差={np.mean(errors):.2f}%"
        
        fig.suptitle(suptitle, fontsize=14, fontweight='bold')
        plt.tight_layout()
        
        # 保存图形
        safe_filename = model_key.replace('/', '_').replace('\\', '_').replace(':', '_')
        output_path = os.path.join(individual_dir, f'{safe_filename}_analysis.png')
        plt.savefig(output_path, dpi=CONFIG['figure_dpi'], bbox_inches='tight', 
                    facecolor='white', edgecolor='none')
        plt.close()
    
    print(f"✓ 已创建 {len(model_errors)} 个单独模型分析图")

def create_summary_statistics(model_errors, model_info, output_dir):
    """创建汇总统计报告"""
    print("\n=== 创建汇总统计报告 ===")
    
    stats_dir = os.path.join(output_dir, 'statistics')
    
    # 创建汇总统计表
    summary_stats = []
    
    for model_key, errors in model_errors.items():
        info = model_info[model_key]
        
        # 基本统计
        basic_stats = {
            'Model_Key': model_key,
            'Method': info['method'],
            'ML_Model': info['ml_model'],
            'K': info['k'],
            'Test_R2': info['test_r2'],
            'Test_MAE': info['test_mae'],
            'Test_RMSLE': info['test_rmsle'],
            'Is_Advanced': info['is_advanced'],
            'Is_Focus_Combination': info['is_focus_combination'],
            'Sample_Count': len(errors),
            'Mean_Relative_Error': np.mean(errors),
            'Median_Relative_Error': np.median(errors),
            'Std_Relative_Error': np.std(errors),
            'Min_Relative_Error': np.min(errors),
            'Max_Relative_Error': np.max(errors),
            'Q25_Relative_Error': np.percentile(errors, 25),
            'Q75_Relative_Error': np.percentile(errors, 75),
            'IQR_Relative_Error': np.percentile(errors, 75) - np.percentile(errors, 25)
        }
        
        # 不同阈值下的频率
        for threshold in CONFIG['error_thresholds']:
            basic_stats[f'Freq_Below_{threshold}pct'] = np.mean(errors <= threshold)
        
        # 特殊统计
        basic_stats['Freq_Above_30pct'] = np.mean(errors > 30)
        basic_stats['Freq_Above_50pct'] = np.mean(errors > 50)
        basic_stats['Excellent_Predictions'] = np.mean(errors <= 5)  # 优秀预测率
        basic_stats['Good_Predictions'] = np.mean(errors <= 10)      # 良好预测率
        basic_stats['Acceptable_Predictions'] = np.mean(errors <= 20) # 可接受预测率
        
        summary_stats.append(basic_stats)
    
    # 转换为DataFrame并排序
    summary_df = pd.DataFrame(summary_stats)
    summary_df = summary_df.sort_values('Test_R2', ascending=False)
    
    # 保存详细统计表
    detailed_stats_file = os.path.join(stats_dir, 'detailed_model_statistics.csv')
    summary_df.to_csv(detailed_stats_file, index=False, encoding='utf-8')
    print(f"✓ 详细统计表已保存: {os.path.basename(detailed_stats_file)}")
    
    # 创建排名统计
    ranking_stats = {
        'Best_R2_Model': summary_df.iloc[0]['Model_Key'],
        'Best_R2_Value': summary_df.iloc[0]['Test_R2'],
        'Best_MAE_Model': summary_df.loc[summary_df['Test_MAE'].idxmin()]['Model_Key'],
        'Best_MAE_Value': summary_df['Test_MAE'].min(),
        'Lowest_Mean_Error_Model': summary_df.loc[summary_df['Mean_Relative_Error'].idxmin()]['Model_Key'],
        'Lowest_Mean_Error_Value': summary_df['Mean_Relative_Error'].min(),
        'Most_Stable_Model': summary_df.loc[summary_df['Std_Relative_Error'].idxmin()]['Model_Key'],
        'Most_Stable_Value': summary_df['Std_Relative_Error'].min(),
        'Best_Excellent_Rate_Model': summary_df.loc[summary_df['Excellent_Predictions'].idxmax()]['Model_Key'],
        'Best_Excellent_Rate_Value': summary_df['Excellent_Predictions'].max(),
        'Total_Models': len(summary_stats),
        'Advanced_Methods_Count': sum(1 for s in summary_stats if s['Is_Advanced']),
        'Focus_Combinations_Count': sum(1 for s in summary_stats if s['Is_Focus_Combination'])
    }
    
    # 保存排名统计
    ranking_file = os.path.join(stats_dir, 'ranking_statistics.txt')
    with open(ranking_file, 'w', encoding='utf-8') as f:
        f.write("=== 模型排名统计 ===\n\n")
        f.write(f"最高R²模型: {ranking_stats['Best_R2_Model']} (R²={ranking_stats['Best_R2_Value']:.4f})\n")
        f.write(f"最低MAE模型: {ranking_stats['Best_MAE_Model']} (MAE={ranking_stats['Best_MAE_Value']:.4f})\n")
        f.write(f"最低平均误差模型: {ranking_stats['Lowest_Mean_Error_Model']} (平均误差={ranking_stats['Lowest_Mean_Error_Value']:.2f}%)\n")
        f.write(f"最稳定模型: {ranking_stats['Most_Stable_Model']} (误差标准差={ranking_stats['Most_Stable_Value']:.2f}%)\n")
        f.write(f"最高优秀预测率模型: {ranking_stats['Best_Excellent_Rate_Model']} (优秀率={ranking_stats['Best_Excellent_Rate_Value']:.3f})\n")
        f.write(f"\n=== 整体统计 ===\n")
        f.write(f"模型总数: {ranking_stats['Total_Models']}\n")
        f.write(f"高级方法模型数: {ranking_stats['Advanced_Methods_Count']}\n")
        f.write(f"重点组合模型数: {ranking_stats['Focus_Combinations_Count']}\n")
        
        # 添加整体性能统计
        f.write(f"\n=== 整体性能统计 ===\n")
        f.write(f"平均R²: {summary_df['Test_R2'].mean():.4f}\n")
        f.write(f"平均MAE: {summary_df['Test_MAE'].mean():.4f}\n")
        f.write(f"平均相对误差: {summary_df['Mean_Relative_Error'].mean():.2f}%\n")
        f.write(f"平均优秀预测率: {summary_df['Excellent_Predictions'].mean():.3f}\n")
        f.write(f"平均良好预测率: {summary_df['Good_Predictions'].mean():.3f}\n")
        f.write(f"平均可接受预测率: {summary_df['Acceptable_Predictions'].mean():.3f}\n")
    
    print(f"✓ 排名统计已保存: {os.path.basename(ranking_file)}")
    
    return summary_df, ranking_stats

def main():
    """主函数"""
    print("=" * 60)
    print("模型预测误差CDF分析程序 - 适配修复版")
    print("=" * 60)
    print(f"输入路径: {CONFIG['input_path']}")
    print(f"输出路径: {CONFIG['output_path']}")
    print(f"时间戳: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 60)
    
    try:
        # 1. 设置输出目录
        output_dir = setup_output_directory()
        
        # 2. 加载数据
        best_models, data_info, features_name = load_model_data()
        
        # 3. 计算误差
        model_errors, model_info, detailed_stats = calculate_model_errors(best_models, data_info)
        
        if not model_errors:
            print("❌ 没有可分析的模型误差数据")
            return
        
        # 4. 保存数据文件
        summary_df = save_data_files(model_errors, model_info, detailed_stats, output_dir)
        
        # 5. 创建综合CDF图
        stats_data = create_comprehensive_cdf_plot(model_errors, model_info, output_dir)
        
        # 6. 创建单独模型分析图
        create_individual_model_plots(model_errors, model_info, detailed_stats, output_dir)
        
        # 7. 创建汇总统计
        summary_df, ranking_stats = create_summary_statistics(model_errors, model_info, output_dir)
        
        # 8. 保存分析配置
        config_file = os.path.join(output_dir, 'analysis_config.txt')
        with open(config_file, 'w', encoding='utf-8') as f:
            f.write("=== CDF分析配置 ===\n")
            f.write(f"输入路径: {CONFIG['input_path']}\n")
            f.write(f"文件后缀: {CONFIG['file_suffix']}\n")
            f.write(f"最大误差阈值: {CONFIG['max_error']}%\n")
            f.write(f"顶级模型数量: {CONFIG['top_models_count']}\n")
            f.write(f"误差阈值列表: {CONFIG['error_thresholds']}\n")
            f.write(f"图片分辨率: {CONFIG['figure_dpi']} DPI\n")
            f.write(f"详细统计: {'启用' if CONFIG['enable_detailed_stats'] else '禁用'}\n")
            f.write(f"分析时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"\n=== Origin Excel文件说明 ===\n")
            f.write(f"00_Origin_Plotting_Guide.xlsx - 包含绘图指南和样式建议\n")
            f.write(f"01_All_Models_CDF_Data.xlsx - 所有模型的CDF数据\n")
            f.write(f"  ├─ CDF_Interpolated_Data: 插值后的统一数据 (推荐用于绘图)\n")
            f.write(f"  ├─ CDF_Raw_Data_Points: 原始CDF数据点\n")
            f.write(f"  └─ Model_Info_and_Stats: 模型信息和统计数据\n")
            f.write(f"02_Top_Models_Comparison.xlsx - 顶级模型对比数据\n")
            f.write(f"  ├─ Top_Models_CDF: 顶级模型CDF数据\n")
            f.write(f"  └─ Top_Models_Stats: 顶级模型统计对比\n")
            f.write(f"03_Error_Distribution_Histograms.xlsx - 误差分布直方图数据\n")
            f.write(f"  ├─ Best_Model_Histogram: 最优模型的误差分布\n")
            f.write(f"  └─ All_Models_Histograms: 所有模型的直方图数据\n")
            f.write(f"04_Radar_Chart_Data.xlsx - 雷达图数据\n")
            f.write(f"  ├─ Radar_Data: 雷达图数值数据\n")
            f.write(f"  └─ Radar_Config: 雷达图配置信息\n")
            f.write(f"individual_models/[model]_analysis.xlsx - 单独模型分析数据\n")
            f.write(f"  ├─ CDF_Data: 单个模型的CDF数据\n")
            f.write(f"  ├─ Histogram_Data: 误差分布数据\n")
            f.write(f"  ├─ Prediction_Scatter: 预测vs实际散点图数据\n")
            f.write(f"  ├─ Perfect_Prediction_Line: 完美预测线数据\n")
            f.write(f"  ├─ Residual_Data: 残差散点图数据\n")
            f.write(f"  ├─ Zero_Reference_Line: 零线参考数据\n")
            f.write(f"  └─ Model_Info: 模型基本信息\n")
            f.write(f"\n=== Origin绘图建议 ===\n")
            f.write(f"1. CDF图: 使用01文件的CDF_Interpolated_Data工作表\n")
            f.write(f"2. 颜色方案: 参考00文件的Style_Suggestions工作表\n")
            f.write(f"3. 线型设置: 高级方法用实线，重点组合用粗线\n")
            f.write(f"4. 坐标轴: X轴0-{CONFIG['max_error']}%，Y轴0-1\n")
            f.write(f"5. 参考线: 在5%, 10%, 20%处添加垂直参考线\n")
        
        print("\n" + "=" * 60)
        print("=== CDF分析完成 ===")
        print("=" * 60)
        print(f"✓ 成功分析 {len(model_errors)} 个模型")
        print(f"✓ 最优模型: {ranking_stats['Best_R2_Model']} (R²={ranking_stats['Best_R2_Value']:.4f})")
        print(f"✓ 所有结果已保存到: {output_dir}")
        
        print(f"\n生成的文件:")
        print(f"📊 figures/comprehensive_cdf_analysis.png - 综合CDF分析图")
        print(f"📊 figures/individual_models/ - 单独模型分析图 ({len(model_errors)}个)")
        print(f"📄 data/model_summary_with_errors.csv - 模型汇总表")
        print(f"📄 data/detailed_statistics.pkl - 详细统计数据")
        print(f"📄 statistics/detailed_model_statistics.csv - 详细统计表")
        print(f"📄 statistics/ranking_statistics.txt - 排名统计")
        print(f"📄 analysis_config.txt - 分析配置")
        print(f"\n📈 Origin绘图专用Excel文件:")
        print(f"📊 excel_for_origin/00_Origin_Plotting_Guide.xlsx - 绘图指南")
        print(f"📊 excel_for_origin/01_All_Models_CDF_Data.xlsx - 所有模型CDF数据")
        print(f"📊 excel_for_origin/02_Top_Models_Comparison.xlsx - 顶级模型对比数据")
        print(f"📊 excel_for_origin/03_Error_Distribution_Histograms.xlsx - 误差分布直方图数据")
        print(f"📊 excel_for_origin/04_Radar_Chart_Data.xlsx - 雷达图数据")
        print(f"📊 excel_for_origin/individual_models/ - 单独模型分析数据 (前5个模型)")
        
        print(f"\n🎯 重点发现:")
        print(f"   最佳R²: {ranking_stats['Best_R2_Value']:.4f} ({ranking_stats['Best_R2_Model']})")
        print(f"   最低MAE: {ranking_stats['Best_MAE_Value']:.4f} ({ranking_stats['Best_MAE_Model']})")
        print(f"   平均优秀预测率(≤5%误差): {summary_df['Excellent_Predictions'].mean():.3f}")
        print(f"   高级方法模型占比: {ranking_stats['Advanced_Methods_Count']}/{ranking_stats['Total_Models']}")
        
        print(f"\n💡 Origin使用说明:")
        print(f"   1. 首先打开 00_Origin_Plotting_Guide.xlsx 查看绘图指南")
        print(f"   2. 主要CDF图使用 01_All_Models_CDF_Data.xlsx")
        print(f"   3. 每个Excel文件包含多个工作表，选择合适的数据工作表")
        print(f"   4. 所有数据已经预处理，可直接用于Origin绘图")
        print(f"   5. 文件包含颜色和样式建议，便于创建专业图表")
        
    except Exception as e:
        print(f"\n❌ 分析过程中发生错误: {e}")
        import traceback
        traceback.print_exc()
        
        # 保存错误日志
        error_log = os.path.join(CONFIG['output_path'], 'error_log.txt')
        os.makedirs(os.path.dirname(error_log), exist_ok=True)
        with open(error_log, 'w', encoding='utf-8') as f:
            f.write(f"错误时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"错误信息: {str(e)}\n\n")
            f.write("详细错误信息:\n")
            f.write(traceback.format_exc())
        print(f"错误日志已保存: {error_log}")

if __name__ == "__main__":
    main()

模型预测误差CDF分析程序 - 适配修复版
输入路径: D:\博一下\manuscript5-变Si组\模型训练_K45_物理引导特征工程_无贝叶斯优化-7.29.3最终_修复版
输出路径: D:\博一下\manuscript5-变Si组\CDF分析8.1
时间戳: 2025-08-01 11:09:28
✓ 输出目录已创建: D:\博一下\manuscript5-变Si组\CDF分析8.1
=== 正在加载模型数据 ===
加载模型文件: D:\博一下\manuscript5-变Si组\模型训练_K45_物理引导特征工程_无贝叶斯优化-7.29.3最终_修复版\best_models_physics_guided_no_bayesian_fixed.pkl
加载数据文件: D:\博一下\manuscript5-变Si组\模型训练_K45_物理引导特征工程_无贝叶斯优化-7.29.3最终_修复版\data_info_physics_guided_no_bayesian_fixed.pkl
加载特征文件: D:\博一下\manuscript5-变Si组\模型训练_K45_物理引导特征工程_无贝叶斯优化-7.29.3最终_修复版\features_name_physics_guided_no_bayesian_fixed.pkl
✓ 成功加载 11 个模型
✓ 数据形状信息: X(221, 40), y(221,)
✓ 特征数量: 40

=== 正在计算模型预测误差 ===
  [1/11] 处理模型: PCC_k=5_GradientBoosting
    ✓ R²=0.9318, MAE=2.0637, 平均相对误差=9.16%
  [2/11] 处理模型: F-Regression_k=5_GradientBoosting
    ✓ R²=0.9197, MAE=2.0991, 平均相对误差=9.16%
  [3/11] 处理模型: RFE_k=4_GradientBoosting
    ✓ R²=0.9177, MAE=2.2332, 平均相对误差=9.44%
  [4/11] 处理模型: Mutual Information_k=5_GradientBoosting
    ✓ R²=0.9157, MAE=2.1871, 平均相对误差=9.29%
 